In [120]:
import pandas as pd
import json
import numpy
import math
from pandas.io.json import json_normalize
from scipy.spatial.distance import cosine
import csv
from osgeo import ogr, gdal
from shapely.geometry import shape
from datetime import datetime
from dateutil.parser import parse


def df_to_geojson(df, properties):
    geojson = {'crs': {'type': 'name', 
                       'properties': {"name": "urn:ogc:def:crs:OGC:1.3:CRS84"}}, 
               'type': 'FeatureCollection', 'features': []}
    for _, row in df.iterrows():
        feature = {'type': 'Feature', 'properties': {}, 
                   'geometry': {'type': 'Polygon', 'coordinates': []}}
        feature['geometry']['coordinates'] = row['geometry.coordinates']
        for prop in properties:
#             print(row[prop])
            if isinstance(row[prop], str):
                feature['properties'][prop] = row[prop]
                continue
            if math.isnan(row[prop] or row[prop]==None):
                feature['properties'][prop] = 0
            else:
                feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [133]:
dates = pd.read_csv("sample_dates.csv")

In [14]:
file = '../../html/geojson.geojson'
with open(file) as f:
    grid = pd.read_json(f, typ='Series')

In [15]:
norm_grid = json_normalize(grid['features'])

In [109]:

norm_grid[['block','parcel']] = norm_grid[['block','parcel']].astype(str)

In [110]:
norm_grid.head()

,geometry.coordinates,geometry.type,block,id,parcel,st_name,st_num,year,type
0,"[[[-82.55622475205546, 35.585252470595705], [-...",Polygon,2,0,2,Southside Ave.,143,1960,Feature
1,"[[[-82.55487708431265, 35.586432936307375], [-...",Polygon,1,2,51,Coxe Ave.,231,1960,Feature
2,"[[[-82.55463127500992, 35.58629677662686], [-8...",Polygon,1,3,50,Southside Ave.,89,1960,Feature
3,"[[[-82.55481165103288, 35.586065196763], [-82....",Polygon,1,4,49,Southside Ave.,101,1960,Feature
4,"[[[-82.55485990777329, 35.586006351274705], [-...",Polygon,1,5,48,Southside Ave.,103,1960,Feature


In [54]:
norm_grid.columns = ['geometry.coordinates','geometry.type','block','id','parcel','st_name','st_num','year','type']

In [137]:
dates[['block','parcel']] = dates[['block','parcel']].astype(str)
dates.head()

,block,parcel,final_price,Prelimiry Title Search,offer_made,offer_accepted,Closed,rejected,Approved for Condemnation,Assigned to Attorney,Petition Filed,Commissioners Hearing,Order Vesting Title,final_title,Assigned to Relocation,Vacated,Assigned for Demolition,removed
0,1,2,8500.0,4/24/69,1969.0,1969.0,6/6/69,NaN,NaN,NaN,NaN,NaN,6/4/69,1969.0,NaN,NaN,NaN,NaN
1,1,3,15000.0,NaN,1969.0,1969.0,NaN,NaN,12/13/70,NaN,7/15/71,6/9/71,6/16/71,1971.0,7/15/71,10/8/71,NaN,NaN
2,1,4,35500.0,11/6/69,1970.0,NaN,7/13/72,NaN,11/9/71,NaN,8/22/72,6/28/72,7/12/72,1972.0,7/13/72,NaN,NaN,NaN
3,1,5,8000.0,10/30/69,1969.0,1974.0,NaN,NaN,NaN,5/8/74,NaN,NaN,5/15/74,1974.0,5/29/74,NaN,NaN,NaN
4,1,6,40000.0,6/6/74,NaN,NaN,NaN,NaN,NaN,9/24/74,NaN,NaN,10/8/74,1974.0,10/14/74,NaN,NaN,NaN


In [135]:
dates.offer_made = pd.to_datetime(pd.Series(dates.offer_made))
dates.offer_accepted = pd.to_datetime(pd.Series(dates.offer_accepted))
dates.rejected = pd.to_datetime(pd.Series(dates.rejected))
dates.final_title = pd.to_datetime(pd.Series(dates.final_title))
dates.removed = pd.to_datetime(pd.Series(dates.removed))

In [136]:
dates.offer_made = dates.offer_made.map(lambda x: x.year)
dates.offer_accepted = dates.offer_accepted.map(lambda x: x.year)
dates.rejected = dates.rejected.map(lambda x: x.year)
dates.final_title = dates.final_title.map(lambda x: x.year)
dates.removed = dates.removed.map(lambda x: x.year)

In [138]:
with_dates = pd.merge(left=norm_grid, right=dates, how='left', on=['block','parcel'])

In [139]:
with_dates.head()

,geometry.coordinates,geometry.type,block,id,parcel,st_name,st_num,year,type,final_price,...,Approved for Condemnation,Assigned to Attorney,Petition Filed,Commissioners Hearing,Order Vesting Title,final_title,Assigned to Relocation,Vacated,Assigned for Demolition,removed
0,"[[[-82.55622475205546, 35.585252470595705], [-...",Polygon,2,0,2,Southside Ave.,143,1960,Feature,48150.0,...,NaN,NaN,NaN,NaN,3/29/76,1976.0,4/5/76,NaN,NaN,NaN
1,"[[[-82.55487708431265, 35.586432936307375], [-...",Polygon,1,2,51,Coxe Ave.,231,1960,Feature,11000.0,...,NaN,NaN,NaN,NaN,12/23/71,1971.0,12/23/71,NaN,6/29/72,1972.0
2,"[[[-82.55463127500992, 35.58629677662686], [-8...",Polygon,1,3,50,Southside Ave.,89,1960,Feature,17000.0,...,NaN,NaN,NaN,NaN,12/31/69,1969.0,12/31/69,NaN,6/29/72,1972.0
3,"[[[-82.55481165103288, 35.586065196763], [-82....",Polygon,1,4,49,Southside Ave.,101,1960,Feature,5700.0,...,NaN,NaN,NaN,NaN,6/12/69,1969.0,6/12/69,8/4/70,8/12/70,1970.0
4,"[[[-82.55485990777329, 35.586006351274705], [-...",Polygon,1,5,48,Southside Ave.,103,1960,Feature,5000.0,...,10/12/70,12/6/71,NaN,NaN,5/27/75,1975.0,5/29/75,NaN,NaN,NaN


In [85]:
# with_dates['id'] = with_dates['properties.id']


In [140]:
cols = ['id','block','parcel','st_num','st_name','final_price','offer_made','offer_accepted','rejected','final_title','removed']
geojson = df_to_geojson(with_dates, cols)

In [141]:
output_filename = 'with_date.geojson'
with open(output_filename, 'w') as output_file:
   json.dump(geojson, output_file, indent=2)

print ("file generated.")

file generated.


In [106]:
type(norm_grid[['parcel']])

pandas.core.frame.DataFrame